In [160]:
import networkx as nx
import numpy as np
import heapq
import unittest
from collections import deque
import matplotlib.pyplot as plt
from queue import Queue

In [161]:
# Graph from previous assignments
graph = dict(
    chi=dict(det=283, cle=345, ind=182),
    cle=dict(chi=345, det=169, col=144, pit=134, buf=189),
    ind=dict(chi=182, col=176),
    col=dict(ind=176, cle=144, pit=185),
    det=dict(chi=283, cle=169, buf=256),
    buf=dict(det=256, cle=189, pit=215, syr=150),
    pit=dict(col=185, cle=134, buf=215, phi=305, bal=247),
    syr=dict(buf=150, phi=253, new=254, bos=312),
    bal=dict(phi=101, pit=247),
    phi=dict(pit=305, bal=101, syr=253, new=97),
    new=dict(syr=254, phi=97, bos=215, pro=181),
    pro=dict(bos=50, new=181),
    bos=dict(pro=50, new=215, syr=312, por=107),
    por=dict(bos=107))

In [162]:
# find distance from source to all other positions in graph, which in this case is the same
# as the distances from every position to the source

def dijkstra(source, graph):
    if source not in graph:
        return None
    visited = set()
    cost_path = {n: float("inf") for n in graph}
    cost_path[source] = (0)
    while len(visited) != len(graph):
        min_node = None
        min_val = float("inf")
        for n in cost_path:
            if n not in visited:
                if cost_path[n] < min_val:
                    min_node = n
        visited.add(min_node)
        for n in graph[min_node]:
            if n not in visited:
                if cost_path[min_node] + graph[min_node][n] < cost_path[n]:
                    cost_path[n] = (cost_path[min_node] + graph[min_node][n])
    return cost_path


In [163]:
# Approach 1: Find the node with the short summed cost from every source node. This approach IS guaranteed to find
# the optimal meeting point, defined as the point that would required the least total travel from all sources, for
# any number of sources (N) in a weighted, connected graph. Note that it expands a full SSSP from every source node.

def meetSum(sources, graph):
    # find distance from sources to all points
    dists_to_sources = [dijkstra(source, graph) for source in sources]
    
    # sum distances from all sources to every node
    total_dists = {g: 0 for g in graph}
    for dist_dict in dists_to_sources:
        for k in dist_dict:
            total_dists[k] += dist_dict[k] 
            
    # choose the node with minimum distance
    meet_point = min(total_dists, key=total_dists.get)
    print("\nMeeting point: " + meet_point)
    for i, source in enumerate(sources):
        print("Distance from %s: %d" % (source, dists_to_sources[i][meet_point]))
    print("Total distance traveled:", total_dists[meet_point])
    return (meet_point, total_dists[meet_point])

In [164]:
meetSum(["det", "col", "syr", "phi"], graph)
meetSum(["det", "syr", "phi"], graph)
meetSum(["det", "syr", "phi", "por", "bos"], graph)
meetSum(["chi", "phi", "por", "bos"], graph)


Meeting point: cle
Distance from det: 169
Distance from col: 144
Distance from syr: 339
Distance from phi: 439
Total distance traveled: 1091

Meeting point: syr
Distance from det: 406
Distance from syr: 0
Distance from phi: 253
Total distance traveled: 659

Meeting point: syr
Distance from det: 406
Distance from syr: 0
Distance from phi: 253
Distance from por: 419
Distance from bos: 312
Total distance traveled: 1390

Meeting point: bos
Distance from chi: 1001
Distance from phi: 312
Distance from por: 107
Distance from bos: 0
Total distance traveled: 1420


('bos', 1420)

In [165]:
# Approach 2: BFS from every source node, terminating when paths from all sources collide. This will find the optimal
# meeting point on unweighted (or unit edge) graphs ONLY. It is NOT guaranteed to find the optimal meeting point on 
# weighted graphs, though it may in some instances.

def meetBFS(sources, graph):
    frontiers = {}
    paths = {}
    for source in sources:
        frontiers[source] = Queue()
        frontiers[source].put(source)
        paths[source] = {source: (0, [source])}
        
    while any(frontiers.values()):
        for source in frontiers.keys():
            node = frontiers[source].get()
            found = True
            for k_path in paths.keys():
                if k_path != source and node not in paths[k_path]:
                    found = False
            if found:
                print("\nMeeting point:", node)
                total = 0
                for s in paths.keys():
                    print("Path from %s to %s: " % (s, node), paths[s][node])
                    total += paths[s][node][0]
                print("Total distance traveled:", total)
                return node
            else:
                for n in graph[node]:
                    if n not in paths[source]:
                        paths[source][n] = (paths[source][node][0] + graph[node][n], paths[source][node][1] + [n])
                        frontiers[source].put(n)
                        
        for key in [k for k in frontiers.keys()]:
            if frontiers[key].empty():
                frontiers.pop(key)
    return None   
    

In [166]:
meetBFS(["det", "col", "syr", "phi"], graph)
meetBFS(["det", "syr", "phi"], graph)
meetBFS(["det", "syr", "phi", "por", "bos"], graph)
meetBFS(["chi", "phi", "por", "bos"], graph)


Meeting point: cle
Path from det to cle:  (169, ['det', 'cle'])
Path from col to cle:  (144, ['col', 'cle'])
Path from syr to cle:  (339, ['syr', 'buf', 'cle'])
Path from phi to cle:  (439, ['phi', 'pit', 'cle'])
Total distance traveled: 1091

Meeting point: cle
Path from det to cle:  (169, ['det', 'cle'])
Path from syr to cle:  (339, ['syr', 'buf', 'cle'])
Path from phi to cle:  (439, ['phi', 'pit', 'cle'])
Total distance traveled: 947

Meeting point: syr
Path from det to syr:  (406, ['det', 'buf', 'syr'])
Path from syr to syr:  (0, ['syr'])
Path from phi to syr:  (253, ['phi', 'syr'])
Path from por to syr:  (419, ['por', 'bos', 'syr'])
Path from bos to syr:  (312, ['bos', 'syr'])
Total distance traveled: 1390

Meeting point: syr
Path from chi to syr:  (689, ['chi', 'det', 'buf', 'syr'])
Path from phi to syr:  (253, ['phi', 'syr'])
Path from por to syr:  (419, ['por', 'bos', 'syr'])
Path from bos to syr:  (312, ['bos', 'syr'])
Total distance traveled: 1673


'syr'

In [170]:
# Approach 2: Dijkstra from every source node, terminating when paths from all sources collide. Again, this is 
# NOT guaranteed to find the optimal meeting point, though it can in some instances.

def meetDijkstra(sources, graph):
    visited = {}
    costs = {}
    paths = {}
    for source in sources:
        visited[source] = set()
        costs[source] = {n: float("inf") for n in graph}
        costs[source][source] = 0
        paths[source] = {source: [source]}
    while (True):
        for source in sources:
            min_node = None
            min_val = float("inf")
            for n in costs[source]:
                if n not in visited[source]:
                    if costs[source][n] < min_val:
                        min_node = n
            visited[source].add(min_node)
            found = True
            for k in sources:
                if k != source and min_node not in visited[k]:
                    found = False
            if found:
                total = 0
                print("\nMeeting Point:", min_node)
                for s in paths.keys():
                    print("Path from %s to %s: " % (s, min_node), (costs[s][min_node], paths[s][min_node]))
                    total += costs[s][min_node]
                print("Total distance traveled:", total)
                return min_node
            for n in graph[min_node]:
                if n not in visited[source]:
                    if costs[source][min_node] + graph[min_node][n] < costs[source][n]:
                        costs[source][n] = (costs[source][min_node] + graph[min_node][n])
                        paths[source][n] = paths[source][min_node] + [n]
            
    

In [173]:
meetDijkstra(["det", "col", "syr", "phi"], graph)
meetDijkstra(["det", "syr", "phi"], graph)
meetDijkstra(["det", "syr", "phi", "por", "bos"], graph)
meetDijkstra(["chi", "phi", "por", "bos"], graph)


Meeting Point: bos
Path from det to bos:  (718, ['det', 'buf', 'syr', 'bos'])
Path from col to bos:  (802, ['col', 'pit', 'phi', 'new', 'bos'])
Path from syr to bos:  (312, ['syr', 'bos'])
Path from phi to bos:  (312, ['phi', 'new', 'bos'])
Total distance traveled: 2144

Meeting Point: bos
Path from det to bos:  (718, ['det', 'buf', 'syr', 'bos'])
Path from syr to bos:  (312, ['syr', 'bos'])
Path from phi to bos:  (312, ['phi', 'new', 'bos'])
Total distance traveled: 1342

Meeting Point: bos
Path from det to bos:  (718, ['det', 'buf', 'syr', 'bos'])
Path from syr to bos:  (312, ['syr', 'bos'])
Path from phi to bos:  (312, ['phi', 'new', 'bos'])
Path from por to bos:  (107, ['por', 'bos'])
Path from bos to bos:  (0, ['bos'])
Total distance traveled: 1449

Meeting Point: bos
Path from chi to bos:  (1001, ['chi', 'det', 'buf', 'syr', 'bos'])
Path from phi to bos:  (312, ['phi', 'new', 'bos'])
Path from por to bos:  (107, ['por', 'bos'])
Path from bos to bos:  (0, ['bos'])
Total distance 

'bos'

In [ ]:
# New idea: Omnidirectional A* Search, terminating when paths from all sources collide, that IS guaranteed to find 
# the optimal meeting point, defined as the point that would required the least total travel from all sources, for 
# any number (N) source nodes in a weighted, connected graph, without necessarily determining the SSSP from all 
# source node.